# DB2-Salesforce connector: Basic user information updates

In [1]:
# Parameters
hours_range = 24*365*30 # number of hours to look back

# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'nanoHUB_user_ID__c'
object_id = 'Contact'

# login parameters to be handled by Papermill
sf_login_params = None 
sql_login_params = None

In [3]:
import sys
sys.path.append(lib_dir)

import pandas as pd
import datetime

## Obtain contacts from DB2 
that have last visit date within range of interest specified by day_range

In [4]:
# Depending on the task, use different query and cutoff

# Hourly update for new registrations
date_cutoff = (datetime.datetime.today().date() - datetime.timedelta(hours=hours_range))\
                    .strftime('%Y-%m-%d')
sql_query = "select id, name, username, block, email, sendEmail, registerDate, lastvisitDate \
                    from jos_users where registerDate >= '%s'" % date_cutoff

# display
print(sql_query)

select id, name, username, block, email, sendEmail, registerDate, lastvisitDate                     from jos_users where registerDate >= '1990-05-27'


In [5]:
# connect with DB2
import sqlalchemy as sql

engine = sql.create_engine('mysql+pymysql://%s:%s@127.0.0.1/nanohub' \
                                               %(sql_login_params['username'], sql_login_params['password']))
df = pd.read_sql_query(sql_query, engine)

# get user profile details
profile_df = pd.read_sql_query("select * from jos_user_profiles where profile_key in ('orgtype', 'organization', 'orcid')", engine)

In [6]:
# display
display(df.head(1))

display(profile_df.head(1))

,id,name,username,block,email,sendEmail,registerDate,lastvisitDate
0,998,hubrepo hubrepo,hubrepo,0,nkissebe@gmail.com,0,2014-11-13 21:09:09,NaT


,id,user_id,profile_key,profile_value,ordering,access
0,1,15623,orgtype,university,2,5


In [7]:
df = pd.merge(df, profile_df[profile_df['profile_key'] == 'orgtype'][['user_id', 'profile_value']],\
                         how='left', left_on='id', right_on='user_id')\
    .drop(columns=['user_id']).rename(columns={'profile_value':'orgtype'})

df = pd.merge(df, profile_df[profile_df['profile_key'] == 'organization'][['user_id', 'profile_value']],\
                         how='left', left_on='id', right_on='user_id')\
    .drop(columns=['user_id']).rename(columns={'profile_value':'organization'})

df = pd.merge(df, profile_df[profile_df['profile_key'] == 'orcid'][['user_id', 'profile_value']],\
                         how='left', left_on='id', right_on='user_id')\
    .drop(columns=['user_id']).rename(columns={'profile_value':'orcid'})

# display
display(df.head(1))

,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization,orcid
0,998,hubrepo hubrepo,hubrepo,0,nkissebe@gmail.com,0,2014-11-13 21:09:09,NaT,NaN,NaN,NaN


In [8]:
# Obtain company domain information from DB2
domain_df = pd.read_sql_query("select name as domain_name, domain, industry, `size range` as size, country \
from wang159_myrmekes.companies_email_domain", engine)

In [9]:
# make sure domain is unique and drop NaN
domain_df = domain_df.drop_duplicates(subset='domain', keep='first').dropna()

In [10]:
# get email domain of nanoHUB users
def get_domain(this_email):
    seg_list = this_email.split('@')
    
    if len(seg_list) == 2:
        return seg_list[1].lower()
    else:
        return None
    
df['email_host'] = df['email'].apply(get_domain)

In [11]:
# get domain by nanoHUB organization field
df['organization'] = df.organization.astype('str').apply(str.lower)

df.loc[(df['organization']=='nan'),'organization'] = None

In [12]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
s_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/users/wang159/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
import re

def clean_domain(this_domain_name):
    
    if not this_domain_name:
        return None
    
    # Replace all non-alphanumeric characters with space
    this_domain_name = re.sub("[^0-9a-zA-Z]+", " ", this_domain_name.lower())
    name_list = this_domain_name.split(' ')
        
    # remove all stop words
    name_list = ['' if x in s_words else x for x in name_list]
    
    return set(filter(None, name_list))

In [14]:
# clean domain name
domain_df['domain_cleaned_set'] = domain_df.domain_name.apply(clean_domain)

# clean nanohub org name
df['org_cleaned_set'] = df.organization.apply(clean_domain)

In [15]:
# hash cleaned domain name
domain_df['domain_cleaned_hash'] = domain_df['domain_cleaned_set'].apply(lambda x: '-'.join(sorted(list(x))) if x else None)

# hash cleaned clean nanohub org name
df['org_cleaned_hash'] = df['org_cleaned_set'].apply(lambda x: '-'.join(sorted(list(x))) if x else None)

## Collaborative filtering for commerical email hosts

In [16]:
def predict_if_commerical(this_email_host_df):
    # predict if this email host is commerical
    top_cnt = this_email_host_df['org_cleaned_hash'].value_counts()
    
    if top_cnt.shape[0] > 0:
        if (top_cnt.iloc[0]/top_cnt.sum()) < 0.2:
            return True
        else:
            return False
    
    else:
        return False
    
is_email_commerical = df[['email_host', 'org_cleaned_hash']].groupby('email_host').apply(predict_if_commerical)

In [17]:
# mark commerical email addresses
df = pd.merge(df, is_email_commerical.reset_index(name='is_email_commerical'),\
         how='left', left_on='email_host', right_on='email_host' ,)

In [18]:
# obtain domain estimation from email addresses
df = pd.merge(df, domain_df[~domain_df.domain.isin(is_email_commerical[is_email_commerical].index)]['domain'],\
         how='left', left_on='email_host', right_on='domain').rename(columns={'domain':'domain_by_email'})

# display
display(df.head(2).T)

,0,1
id,998,1683
name,hubrepo hubrepo,nanoHUB support
username,hubrepo,support
block,0,0
email,nkissebe@gmail.com,support@nanohub.org
sendEmail,0,0
registerDate,2014-11-13 21:09:09,2008-11-19 22:51:04
lastvisitDate,NaT,2008-11-19 23:55:30
orgtype,NaN,university
organization,None,purdue university


## Fuzzy derive organization from nanoHUB profiles

In [19]:
# attempt direct join by hash
domain_subset_df = domain_df[domain_df.domain.isin(df.email_host.unique())][['domain', 'domain_cleaned_hash']]
domain_subset_df = domain_subset_df[~domain_subset_df.domain.isna()&~domain_subset_df.domain_cleaned_hash.isna()]

derived_df = pd.merge(df, domain_subset_df.rename(columns={'domain':'domain_by_profile'}),\
         how='left', left_on='org_cleaned_hash', right_on='domain_cleaned_hash')\
        .drop('domain_cleaned_hash', axis=1)

# display
derived_df.sample(5).T

,29935,61327,62627,171327,82750
id,33328,64679,65947,203755,88613
name,Kari Driemeyer,Joe Hagan,Eadaoin McClean,Kartikeya Dixit,Vishnu Kumar
username,kdriemeyer,hagjos,iodine,kartikdiitk,vish7908
block,0,0,0,0,0
email,karidriemeyer2012@u.northwestern.edu,hagjos@ksu.edu,eadaoin.mc-clean@ucdconnect.ie,kartikdiitk@gmail.com,vish7908@gmail.com
sendEmail,0,0,1,-1,-1
registerDate,2009-01-16 01:31:13,2012-02-26 21:35:01,2012-04-02 17:04:47,2018-07-18 16:59:14,2013-09-27 02:16:51
lastvisitDate,2009-03-05 22:47:41,2012-02-29 03:53:49,2013-07-25 13:01:54,2018-07-18 16:59:15,2013-09-27 02:17:23
orgtype,university,universityundergraduate,universitygraduate,universitygraduate,NaN
organization,northwestern university,None,university college dublin,None,None


In [20]:
# attempt to reduce domain_df size to speedup Jaccard calculation
from itertools import chain

all_word_set = set(chain.from_iterable(derived_df[(derived_df['domain_by_profile'].isna() & (~derived_df['organization'].isna()))]\
                    .org_cleaned_set.values))

# select the domain_df entry with words from all_word_set
domain_subset_df = domain_df[domain_df.domain_cleaned_set.notnull()]
domain_subset_df = domain_subset_df[domain_subset_df.domain_cleaned_set\
                             .apply(lambda x: True if len(x.intersection(all_word_set)) > 0 else False)]

In [21]:
def get_org(this_df):
    # look at the cohort with this label, do they have a commonly agreed domain_by_email? If so, use it
    this_dbe_count = this_df.domain_by_email.value_counts()

    if this_dbe_count.shape[0] < 1:
        return None

    this_top_dbe_perc = this_dbe_count.iloc[0]/this_dbe_count.sum()

    # if top choice occupies majority of cases, then this cohort can be implied to this domain_by_email address
    if this_top_dbe_perc > 0.5:
        return this_dbe_count.index[0] 


derived_hash = derived_df.groupby('org_cleaned_hash').apply(get_org)

In [22]:
derived_df = pd.merge(derived_df, derived_hash.reset_index(name='domain_by_profile_infer')\
                      , how='left', left_on='org_cleaned_hash', right_on='org_cleaned_hash')

In [81]:
# calculate a composite domain based on domain_by_email, domain_by_profile, and domain_by_profile_infer
def get_composite_domain(this_df):

    # if domain found via profile with direct match, use it
    if this_df.domain_by_profile != '':
        return this_df.domain_by_profile
    
    # if domain found via email, use it
    if this_df.domain_by_email != '':
        return this_df.domain_by_email    
    
    # if domain found via profile but not with a direct match, use it
    if this_df.domain_by_profile_infer != '':
        return this_df.domain_by_profile_infer
    
    # else, no organization can be concluded, return none
    return None

    
derived_df['domain_final'] = derived_df[['domain_by_email', 'domain_by_profile', 'domain_by_profile_infer']].fillna('')\
                                                        .apply(get_composite_domain, axis=1)

## Update Salesforce institution 

In [83]:
# get a unique list of organizations that appears in contact
#org_df = derived_df[['domain_name', 'domain', 'industry', 'size', 'country']].drop_duplicates().dropna()
org_df = pd.merge(derived_df[['domain_final']].drop_duplicates().dropna(), domain_df, how='inner', left_on='domain_final', right_on='domain') 
org_df = org_df[['domain_name', 'domain', 'industry', 'size', 'country']]

# display
display(org_df.head(2))

,domain_name,domain,industry,size,country
0,purdue university,purdue.edu,higher education,10001+,united states
1,netzero inc,netzero.net,computer software,1-10,united states


In [84]:
df_sf = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here
df_sf['Name'] = org_df['domain_name']

df_sf['Size_Range__c'] = org_df['size'].fillna('Unknown')
df_sf['Country__c'] = org_df['country']
df_sf['Domain__c'] = org_df['domain']
df_sf['Industry__c'] = org_df['industry']

sf_original_fields = df_sf.columns

# display
df_sf.head(2).T

,0,1
Name,purdue university,netzero inc
Size_Range__c,10001+,1-10
Country__c,united states,united states
Domain__c,purdue.edu,netzero.net
Industry__c,higher education,computer software


In [85]:
from DB2SalesforceAPI import DB2SalesforceAPI

# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

db_s.object_id = 'organization__c'
db_s.external_id = 'Domain__c'

Obtained Salesforce access token ...... True


In [86]:
# send data to Salesforce
db_s.send_data(df_sf)

[Success] Bulk job creation successful. Job ID = 7505w00000M3DxgAAF
[Success] CSV upload successful. Job ID = 7505w00000M3DxgAAF
[Success] Closing job successful. Job ID = 7505w00000M3DxgAAF


In [89]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000M3DxgAAF',
 'operation': 'upsert',
 'object': 'organization__c',
 'createdById': '0055w00000ArpYvAAJ',
 'createdDate': '2020-05-20T00:44:41.000+0000',
 'systemModstamp': '2020-05-20T00:45:02.000+0000',
 'state': 'JobComplete',
 'externalIdFieldName': 'Domain__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 3367,
 'numberRecordsFailed': 6,
 'retries': 0,
 'totalProcessingTime': 5762,
 'apiActiveProcessingTime': 4707,
 'apexProcessingTime': 0}

In [90]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_failed_results())

('"sf__Id","sf__Error",Country__c,Domain__c,Industry__c,Name,Size_Range__c\n'
 '"","STRING_TOO_LONG:Organization: data value too large: centro nacional de '
 'energÃ\xadas renovables (cener) - national renewable energy centre (max '
 'length=80):Name --","spain","cener.com","renewables & environment","centro '
 'nacional de energÃ\xadas renovables (cener) - national renewable energy '
 'centre","201-500"\n'
 '"","STRING_TOO_LONG:Organization: data value too large: department '
 'werkstoffwissenschaften der friedrich-alexander-universitÃ¤t '
 'erlangen-nÃ¼rnberg (max length=80):Name '
 '--","germany","fau.de","research","department werkstoffwissenschaften der '
 'friedrich-alexander-universitÃ¤t erlangen-nÃ¼rnberg","1-10"\n'
 '"","STRING_TOO_LONG:Organization: data value too large: consortium of '
 'universities for the advancement of hydrologic science, inc. (cuahsi) (max '
 'length=80):Name --","united states","cuahsi.org","research","consortium of '
 'universities for the advancement

## Obtain newly updated organization IDs from Salesforce

In [91]:
# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

Obtained Salesforce access token ...... True


In [92]:
# get Salesforce ID for organizations
sf_org_ID_df = db_s.query_data('SELECT Id, Domain__c FROM organization__c')

# display
sf_org_ID_df.head(3)

[Success] Bulk job creation successful. Job ID = 7505w00000M3DzNAAV
{"id":"7505w00000M3DzNAAV","operation":"query","object":"organization__c","createdById":"0055w00000ArpYvAAJ","createdDate":"2020-05-20T00:46:25.000+0000","systemModstamp":"2020-05-20T00:46:26.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000M3DzNAAV","operation":"query","object":"organization__c","createdById":"0055w00000ArpYvAAJ","createdDate":"2020-05-20T00:46:25.000+0000","systemModstamp":"2020-05-20T00:46:32.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":3463,"retries":0,"totalProcessingTime":323}
[Success] Bulk job completed successfully.


,Domain__c,Id
0,upenn.edu,a0r5w00000V42c0AAB
1,fer.hr,a0r5w00000V42c1AAB
2,wheatoncollege.edu,a0r5w00000V42c2AAB


In [93]:
# join salesforce ID back to contact DF
df = pd.merge(sf_org_ID_df, derived_df, how='right', left_on='Domain__c', right_on='domain_final')\
            .drop('Domain__c', axis=1)\
            .rename(columns={'Id':'Salesforce_org_ID'})

In [94]:
# display
display(df.head(2))

,Salesforce_org_ID,id,name,username,block,email,sendEmail,registerDate,lastvisitDate,orgtype,organization,orcid,email_host,org_cleaned_set,org_cleaned_hash,is_email_commerical,domain_by_email,domain_by_profile,domain_by_profile_infer,domain_final
0,a0r5w00000V42c0AAB,7848,Matyas Matlo,matlo,0,matlo@seas.upenn.edu,0,2004-12-13 11:23:29,2004-12-20 13:40:42,NaN,university of pennsylvania,NaN,seas.upenn.edu,"{university, pennsylvania}",pennsylvania-university,False,NaN,upenn.edu,upenn.edu,upenn.edu
1,a0r5w00000V42c0AAB,8760,Michael Fischbein,mlfisch3,0,mlfisch3@sas.upenn.edu,0,2005-04-05 19:37:53,2005-04-05 19:37:53,NaN,university of pennsylvania,NaN,sas.upenn.edu,"{university, pennsylvania}",pennsylvania-university,False,NaN,upenn.edu,upenn.edu,upenn.edu


## Match data with Salesforce format

In [95]:
# split full name into first, middle, and last names
def split_full_name(this_name):
    this_name_list = list(filter(None, this_name.split(' ')))
    
    if len(this_name_list) == 1:
        # single word name
        return pd.Series([this_name_list[0],None, this_name_list[0]])
    
    elif len(this_name_list) > 1:
        # multi word name
        return pd.Series([this_name_list[0],' '.join(this_name_list[1:-1]),this_name_list[-1]])

In [96]:
df_sf = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here
df_sf[['firstname', 'Middle_name__c', 'lastname']] = df['name'].apply(split_full_name)

df_sf['nanoHUB_user_ID__c']            = df['id']
df_sf['nanoHUB_username__c']           = df['username']
df_sf['Email']                         = df['email'].fillna('').apply(lambda x: '' if '@invalid' in x else x)

# for sendEmail: 0 = opt-out, 1 = receive email. For salesforce HasOptedOutOfEmail, it's exact opposite
df_sf['HasOptedOutOfEmail']            = df['sendEmail'].apply(lambda x: 0 if x==1 else 1)
df_sf['nanoHUB_account_BLOCKED__c']    = df['block'].fillna(0)

# solidify time-related columns from datetime to string
df_sf['nanoHUB_registration_date__c']  = df['registerDate'].dt.date.fillna('').astype('str')
df_sf['nanoHUB_last_active_date__c']   = df['lastvisitDate'].dt.date.fillna('').astype('str')

# Tableau detailed view
df_sf['Detailed_user_timeline_to_Tableau__c'] = df_sf['Email'].apply(lambda x: 'https://tableauqa.itap.purdue.edu/views/profile/ProfileTimeline?Id%20Email='+x+'\
&:iframeSizedToWindow=true&:embed=y&:showAppBanner=false\
&:display_count=no&:showVizHome=no#6' if x != '' else '')

# nanoHUB user profile
df_sf['nanoHUB_user_page__c'] = df_sf['nanoHUB_user_ID__c'].apply(lambda x: 'https://nanohub.org/members/%d'%x if x != '' else '')

df_sf['ORCID__c'] = df['orcid'].fillna('')
df_sf['Organization__c'] = df['organization'].fillna('')

# derived information
df_sf['Organization_email_derived__c'] = df['Salesforce_org_ID']

sf_original_fields = df_sf.columns

# display
df_sf.head(2).T

,0,1
firstname,Matyas,Michael
Middle_name__c,,
lastname,Matlo,Fischbein
nanoHUB_user_ID__c,7848,8760
nanoHUB_username__c,matlo,mlfisch3
Email,matlo@seas.upenn.edu,mlfisch3@sas.upenn.edu
HasOptedOutOfEmail,1,1
nanoHUB_account_BLOCKED__c,0,0
nanoHUB_registration_date__c,2004-12-13,2005-04-05
nanoHUB_last_active_date__c,2004-12-20,2005-04-05


## To Salesforce Sales Cloud CRM

In [97]:
from DB2SalesforceAPI import DB2SalesforceAPI

# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

db_s.object_id = object_id
db_s.external_id = external_id

Obtained Salesforce access token ...... True


In [98]:
# send data to Salesforce
db_s.send_data(df_sf)

[Success] Bulk job creation successful. Job ID = 7505w00000M3E4IAAV
[Success] CSV upload successful. Job ID = 7505w00000M3E4IAAV
[Success] Closing job successful. Job ID = 7505w00000M3E4IAAV


In [118]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000M3E4IAAV',
 'operation': 'upsert',
 'object': 'Contact',
 'createdById': '0055w00000ArpYvAAJ',
 'createdDate': '2020-05-20T00:49:58.000+0000',
 'systemModstamp': '2020-05-20T00:58:44.000+0000',
 'state': 'JobComplete',
 'externalIdFieldName': 'nanoHUB_user_ID__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 218651,
 'numberRecordsFailed': 200,
 'retries': 0,
 'totalProcessingTime': 1934037,
 'apiActiveProcessingTime': 1726931,
 'apexProcessingTime': 22490870}

In [119]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_failed_results())

('"sf__Id","sf__Error",Detailed_user_timeline_to_Tableau__c,Email,HasOptedOutOfEmail,Middle_name__c,ORCID__c,Organization__c,Organization_email_derived__c,firstname,lastname,nanoHUB_account_BLOCKED__c,nanoHUB_last_active_date__c,nanoHUB_registration_date__c,nanoHUB_user_ID__c,nanoHUB_user_page__c,nanoHUB_username__c\n'
 '"","STRING_TOO_LONG:Organization (from nanoHUB Profile): data value too '
 'large: deptt of applied physics, shri shankaracharya college of engineering  '
 '&amp; technology-junwani bhilai (chhattisgarh) (max '
 'length=100):Organization__c '
 '--","https://tableauqa.itap.purdue.edu/views/profile/ProfileTimeline?Id%20Email=drmohanlv@yahoo.com&:iframeSizedToWindow=true&:embed=y&:showAppBanner=false&:display_count=no&:showVizHome=no#6","drmohanlv@yahoo.com","false","Mohan '
 'L","","deptt of applied physics, shri shankaracharya college of engineering  '
 '& technology-junwani bhilai '
 '(chhattisgarh)","","Dr","Verma","false","2008-04-02","2008-02-06","26563.0","https://